In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
%pip install gensim
%pip install nltk

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joelrudsberg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [66]:
class Data(Enum):
    MOVIES = "movie.metadata.tsv"
    PLOTS = "plot_summaries.txt"
    NAME_CLUSTERS = "name.clusters.txt"

def load_data(type : Data, columns, index_col = ""):
    return pd.read_csv(
        f'./rawdata/MovieSummaries/{type.value}', 
        sep="\t", 
        names=columns,
        index_col=index_col if index_col != "" else columns[0]
    )

movie_df = load_data(
    Data.MOVIES, 
    ["wiki_id", "freebase_id", "name", "release_date", "revenue", "runtime", "languages", "countries", "genres"],
)
plot_df = load_data(
    Data.PLOTS, 
    ["wiki_id", "plot"]
)
name_df = load_data(
    Data.NAME_CLUSTERS, 
    ["character_name", "freebase_id"],
    index_col="freebase_id"
)

In [67]:
import re

# Parses the multi value structure of the dataset format into a sorted array of items
# Returns null values for empty lists as NaN (so all missing values we use are NaN)
def parse_multi_values(x): 
    if not x: 
        return []
    x = x.split(',')
    result = []
    for combo in x: 
        combo = combo.split(":")
        if len(combo) > 1: 
            combo[1] = re.sub("[^a-zA-Z0-9\s]+", "", combo[1])
            combo[1] = re.sub("^\s", "", combo[1])
            combo[1] = combo[1].strip()
            result.append(combo[1])
    if not result:
        return []
    return sorted(result)

# Add plot summaries to the movie df
movie_df = movie_df.join(plot_df)

# Parses the language column specifically by removing "Langauge" from each entry
def parse_language(languages):
    if not languages or type(languages) != list:
        return []
    return list(map(lambda l : l.replace("Language", "").strip(), languages))

# Parses the date format into an int representing only the year. Values with missing entries are given NaN.
def parse_date(x):
    if not x or x == np.nan:
        return np.NaN
    return str(x).split("-")[0]

# Parse multi value attributes into arrays
for attribute in ["languages", "countries", "genres"]:
    movie_df[attribute] = movie_df[attribute].apply(parse_multi_values)

# Remove "Language" from language column
movie_df["languages"] = movie_df["languages"].apply(parse_language)

# Parse release-date to all use the year as an int (don't need increased granularity right now)
movie_df["release_date"] = movie_df["release_date"].apply(parse_date)

# Replace missing numeric values with zero and convert to int rather than float
for col in ["runtime", "revenue"]:
    movie_df[col] = movie_df[col].fillna(0)
    movie_df[col] = movie_df[col].astype(int)

movie_df

,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,plot
wiki_id,,,,,,,,,
975900,/m/03vyhn,Ghosts of Mars,2001,14010832,98,[English],[United States of America],"[Action, Adventure, Horror, Science Fiction, S...","Set in the second half of the 22nd century, th..."
3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000,0,95,[English],[United States of America],"[Biographical film, Crime Drama, Drama, Mystery]",NaN
28463795,/m/0crgdbh,Brun bitter,1988,0,83,[Norwegian],[Norway],"[Crime Fiction, Drama]",NaN
9363483,/m/0285_cd,White Of The Eye,1987,0,110,[English],[United Kingdom],"[Erotic thriller, Psychological thriller, Thri...",A series of murders of rich young women throug...
261236,/m/01mrr1,A Woman in Flames,1983,0,106,[German],[Germany],[Drama],"Eva, an upper class housewife, becomes frustra..."
...,...,...,...,...,...,...,...,...,...
35228177,/m/0j7hxnt,Mermaids: The Body Found,2011,0,120,[English],[United States of America],[Drama],Two former National Oceanic Atmospheric Admini...
34980460,/m/0g4pl34,Knuckle,2011,0,96,[English],"[Ireland, United Kingdom]","[Biographical film, Documentary, Drama]",{{No plot}} This film follows 12 years in the ...
9971909,/m/02pygw1,Another Nice Mess,1972,0,66,[English],[United States of America],"[Comedy, Satire]",NaN


In [79]:
import string

def parseDocument(doc):
        # TODO: probably don't need stuff to remove puctuation eg since that should be handled by word_tokenizer
        return word_tokenize(doc.lower().replace('\n', ' ').replace('\r', '').translate(str.maketrans('', '', string.punctuation)))

# Tokenize all plots 
tokenized_df = plot_df.copy() 
tokenized_summaries = tokenized_df['plot'].values
tokenized_summaries = [TaggedDocument(parseDocument(doc), [idx]) for idx, doc in enumerate(tokenized_summaries)]


freebase_id                                            /m/02r1ggm
name                                         Japanil Kalyanaraman
release_date                                                 1985
revenue                                                         0
runtime                                                         0
languages                                                 [Tamil]
countries                                                 [India]
genres                               [Action, Comedy film, Drama]
plot            Raman  is a journalist, who exposes a smugglin...
wiki_id                                                  11143089
Name: 19136, dtype: object

In [57]:
import pathlib
import multiprocessing

# Loads existing model from disk if available else trains new model
# Set force_train_new to True to train a new model even if a previous model exists
def create_doc2vec_model(documents, force_train_new = False):
    fname = str(pathlib.Path().resolve()) + "/doc2vec_model"
    if not force_train_new:
        # Try to load, if it succeeds then previous model existed so return that one
        try:
            return Doc2Vec.load(fname)
        except:
            print("Failed to load existing model")

    # Train model
    print("Start training of Doc2Vec model...")
    model = Doc2Vec(
        documents, 
        window=2, 
        min_count=1, 
        workers=multiprocessing.cpu_count(), 
        epochs = 100
    )
    
    # Persist the model
    print(f"Finished training Doc2Vec, saving it to file {fname}")
    model.save(fname)
    return model

# Train the doc2vec model
model = create_doc2vec_model(tokenized_summaries, force_train_new=False)

Start training of Doc2Vec model...
Finished training Doc2Vec, saving it to file /Users/joelrudsberg/Desktop/EPFL/Applied Data Analysis/ada-2022-project-adawizardry/doc2vec_model


In [74]:
apollo_wiki = """Apollo 13 (April 11–17, 1970) was the seventh crewed mission in the Apollo space program and the third meant to land on the Moon. The craft was launched from Kennedy Space Center on April 11, 1970, but the lunar landing was aborted after an oxygen tank in the service module (SM) failed two days into the mission. The crew instead looped around the Moon and returned safely to Earth on April 17. The mission was commanded by Jim Lovell, with Jack Swigert as command module (CM) pilot and Fred Haise as Lunar Module (LM) pilot. Swigert was a late replacement for Ken Mattingly, who was grounded after exposure to rubella.

A routine stir of an oxygen tank ignited damaged wire insulation inside it, causing an explosion that vented the contents of both of the SM's oxygen tanks to space. Without oxygen, needed for breathing and for generating electric power, the SM's propulsion and life support systems could not operate. The CM's systems had to be shut down to conserve its remaining resources for reentry, forcing the crew to transfer to the LM as a lifeboat. With the lunar landing canceled, mission controllers worked to bring the crew home alive.

Although the LM was designed to support two men on the lunar surface for two days, Mission Control in Houston improvised new procedures so it could support three men for four days. The crew experienced great hardship, caused by limited power, a chilly and wet cabin and a shortage of potable water. There was a critical need to adapt the CM's cartridges for the carbon dioxide scrubber system to work in the LM; the crew and mission controllers were successful in improvising a solution. The astronauts' peril briefly renewed public interest in the Apollo program; tens of millions watched the splashdown in the South Pacific Ocean on television.

An investigative review board found fault with preflight testing of the oxygen tank and Teflon being placed inside it. The board recommended changes, including minimizing the use of potentially combustible items inside the tank; this was done for Apollo 14. The story of Apollo 13 has been dramatized several times, most notably in the 1995 film Apollo 13 – based on Lost Moon, the 1994 memoir co-authored by Lovell – and an episode of the 1998 miniseries From the Earth to the Moon."""
test_doc = parseDocument(apollo_wiki)
test_vector = model.infer_vector(test_doc)

top_similar = model.dv.most_similar(positive=[test_vector],topn=100)
# Reminder, id here is the natural index from movie_df
top_similar_df = pd.DataFrame(top_similar, columns=["id", "similarity"]).set_index("id")

movie_df = movie_df.reset_index(drop=True)

top_similar_df = pd.merge(top_similar_df, movie_df, left_index=True, right_index=True)[["name", "similarity", "release_date", "plot"]]
top_similar_df

# top_similar_df = 

# top_similar_df = top_similar_df.set_index("wiki_id")
# top_similar_df.head(n=30)

# Get similarity for all wiki pages of events and save results to new with the similarity score 

# (event, year)
# (movie, similarity)
# Has the event caused a change in the *trend*?
# Approaches:
# * avg per similarity per year? -> hard to find change, diluted by other events?
# * look at movies with similarity score over some threshold to find all movies that are somewhat similar and do line plot of those movies
# # -> downside is that a very similar movie and not similar movie is weighted equally. -> use similarity as weight?
# * 


,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,plot,wiki_id
0,/m/03vyhn,Ghosts of Mars,2001,14010832,98,[English],[United States of America],"[Action, Adventure, Horror, Science Fiction, S...","Set in the second half of the 22nd century, th...",975900
1,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000,0,95,[English],[United States of America],"[Biographical film, Crime Drama, Drama, Mystery]",NaN,3196793
2,/m/0crgdbh,Brun bitter,1988,0,83,[Norwegian],[Norway],"[Crime Fiction, Drama]",NaN,28463795
3,/m/0285_cd,White Of The Eye,1987,0,110,[English],[United Kingdom],"[Erotic thriller, Psychological thriller, Thri...",A series of murders of rich young women throug...,9363483
4,/m/01mrr1,A Woman in Flames,1983,0,106,[German],[Germany],[Drama],"Eva, an upper class housewife, becomes frustra...",261236
...,...,...,...,...,...,...,...,...,...,...
81736,/m/0j7hxnt,Mermaids: The Body Found,2011,0,120,[English],[United States of America],[Drama],Two former National Oceanic Atmospheric Admini...,35228177
81737,/m/0g4pl34,Knuckle,2011,0,96,[English],"[Ireland, United Kingdom]","[Biographical film, Documentary, Drama]",{{No plot}} This film follows 12 years in the ...,34980460
81738,/m/02pygw1,Another Nice Mess,1972,0,66,[English],[United States of America],"[Comedy, Satire]",NaN,9971909
81739,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992,0,150,[Japanese],[Japan],"[Adventure, Animation, Anime, Drama, Japanese ...","The story takes place in the year 2092,The Sup...",913762


,similarity
id,
39117,0.696025
19136,0.675214
3295,0.633047
11877,0.627408
38563,0.610541
...,...
10350,0.493853
19960,0.492082
23571,0.491673


,name,similarity,release_date,plot
39117,13 Beloved,0.696025,2006,"{{Plot|section|date""4"" cellpadding""width:30%; ..."
19136,Japanil Kalyanaraman,0.675214,1985,"Raman is a journalist, who exposes a smugglin..."
3295,The Rifleman of the Voroshilov Regiment,0.633047,1999,World War II veteran Ivan Afonin lives with hi...
11877,I Like Your Nerve,0.627408,1931,NaN
38563,What's Wrong With Virginia,0.610541,2010,"In a small town, mentally unstable Virginia ha..."
...,...,...,...,...
10350,Nutcracker: The Motion Picture,0.493853,1986,NaN
19960,Swanee River,0.492082,1939,The family of Stephen Foster insists that he ...
23571,Herostratus,0.491673,1967,NaN
26102,Acting Hamlet in the Village of Mrdusa Donja,0.491381,1974,NaN


In [78]:
index = movie_df.iloc[19136].name
movie_df.loc[index]

freebase_id                                            /m/02r1ggm
name                                         Japanil Kalyanaraman
release_date                                                 1985
revenue                                                         0
runtime                                                         0
languages                                                 [Tamil]
countries                                                 [India]
genres                               [Action, Comedy film, Drama]
plot            Raman  is a journalist, who exposes a smugglin...
wiki_id                                                  11143089
Name: 19136, dtype: object